# Exercise 4 - A practical case
## I. Abstract
todo
## II. Introduction
todo
## III. Specification of the problem and organization of the software
<p>We are the new data analyst in a Telecom company. Our first project is to try to reduce the
churn of our new service. Modeling churn means to understand what keeps the customer engaged
to our product. As an analyst, our goal is to predict or describe the churn rate i.e. the rate at
which customer leave or cease the subscription to a service. Its value lies in the fact that engaging
new customers is often more costly than retaining existing ones. For that reason subscription
business-based companies usually have proactive policies towards customer retention.</p>
<p>In this case study, we aim at building a machine learning model for customer churn prediction
on data from a Telecom company. Each row on the dataset represents a subscribing telephone
customer. Each column contains customer attributes such as phone number, call minutes used
during different times of day, charges incurred for services, lifetime account duration, and whether
or not the customer is still a customer. Data can be found in the file: churn.csv.</p>

The complete set of attributes is the following:
<ul>
<li>State: categorical, for the 50 states and the District of Columbia</li>
<li>Account length: integer-valued, how long an account has been active</li>
<li>Area code: categorical</li>
<li>Phone number: customer ID</li>
<li>International Plan: binary feature, yes or no</li>
<li>VoiceMail Plan: binary feature, yes or no</li>
<li>Number of voice mail messages: integer-valued</li>
<li>Total day minutes: continuous, minutes customer used service during the day</li>
<li>Total day calls: integer-valued</li>
<li>Total day charge: continuous</li>
<li>Total evening minutes: continuous, minutes customer used service during the evening</li>
<li>Total evening calls: integer-valued</li>
<li>Total evening charge: continuous</li>
<li>Total night minutes: continuous, minutes customer used service during the night</li>
<li>Total night calls: integer-valued</li>
<li>Total night charge: continuous</li>
<li>Total international minutes: continuous, minutes customer used service to make international
calls</li>
<li>Total international calls: integer-valued</li>
<li>Total international charge: continuous</li>
<li>Number of calls to customer service: integer-valued</li>
</ul>

## IV. Experiments

### Code Design
All classifiers must be coded from scratch in python, no toolkits are allowed to be used
except for numpy and CVX (http://www.cvxpy.org/). All classifiers have to be fully integrable
in the scikit-learn (sklearn) ecosystem, i.e. inherit from BaseEstimator and implement the basic
methods, fit, predict, score, ... You can check how to integrate your classifier in http://scikit-learn.org/stable/developers/contributing.html#coding-guidelines. We do not need to pass all unit
testing but at least we would like to use several of the nice functions such as gridsearchCV, etc
with your estimator.

<div class = "alert alert-success" style = "border-radius:10px">**WP 1**
<ol>
<li> Read the dataset ‘churn.csv‘ and split it into features and labels</li> 
<li> Convert data to floating point numbers </li>
<li> Drop all not useful features </li>
</ol>
<div>

<div class = "alert alert-danger" style="border-radius:10px"> **HINT:** You may want to consider
pandas library for cleaning purposes.
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the dataset ‘churn.csv‘
data = pd.read_csv('churn.csv')

In [2]:
# clean data
# drop features: state, account length, area code, phone
to_drop = ['State', 'Account Length', 'Area Code', 'Phone']
clean_data = data.drop(to_drop, axis=1)

boolean_cols = ["Int'l Plan","VMail Plan", 'Churn?']
for col in boolean_cols:
    # convert categorical data into numeric data
    clean_data[col] = clean_data[col].astype('category').cat.codes

# convert to float data
clean_data = clean_data.astype(np.float)
# split it into features and labels
X, y = clean_data.drop(['Churn?'], axis=1), clean_data['Churn?']

# set y values as 1 and -1
y[y == 0] = -1

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# TODO: can we use this function? 
# normalize data
X = StandardScaler().fit_transform(X)

# TODO: can we use this function? IMPORTANT: scikit-learn >= 18.0 
# split into training and testing datasets
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.2, random_state=42)

<div class = "alert alert-success" style = "border-radius:10px">**WP 2**
<ol>
<li> Try solving the problem using sklearn techniques.</li> 
</ol>
<div>

In [4]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier().fit(X_train, y_train)
print('Random Forest (n_estimators=10): {:.3f}'.format(clf.score(X_test, y_test)))

Random Forest (n_estimators=10): 0.940


<div class = "alert alert-success" style = "border-radius:10px">**WP 3**
<ol>
<li> Implement your classifier.</li> 
</ol>
<div>

In [5]:
# include BaseEstimator class and for generating random numbers according to scikit guidelines
import sklearn

# Calculate the Gini index for a split dataset
def weighted_gini_index(groups):
    gini = 0.0
    for group in groups:
        labels = group[:, -1]
        if len(labels):
            class_values, counts = np.unique(labels, return_counts=True)
            class_prob = counts / float(len(labels))
            weight = 1.0 * len(group) / len(groups)
            gini += weight * (1.0 - np.dot(class_prob, class_prob))
    return gini

# Calculate the Entropy for a split dataset
def weighted_entropy(groups):
    entropy = 0.0
    for group in groups:
        labels = group[:, -1]
        if len(labels):
            class_values, counts = np.unique(labels, return_counts=True)
            class_prob = counts / float(len(labels))
            weight = 1.0 * len(group) / len(groups)
            entropy += -weight * np.dot(class_prob, np.log2(class_prob))
    return entropy

class MyDecisionTreeClassifier(sklearn.base.BaseEstimator):
    def __init__(self, max_features=None, max_depth=None, min_samples_leaf=1, boostrap=False, criterion='gini', random_state=None):
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.boostrap = boostrap
        self.criterion = criterion
        self.random_state = random_state
        
        #internal representation
        self._num_features = None
        self._score_f = None
        self._tree = None
                
    def fit(self, X, y):
        # generate random numbers according to scikit guidelines        
        self.random_state = sklearn.utils.check_random_state(self.random_state)
        
        if self.criterion == 'gini':
            self._score_f = weighted_gini_index
        elif self.criterion == 'entropy':
            self._score_f = weighted_entropy
        else:
            self._score_f = weighted_gini_index
        
        # default: all of features
        self._num_features = len(X[0]) if self.max_features is None else self.max_features
        
        dataset = np.column_stack((X, y))

        # TODO: when boostrap is false, should data be shuffled?
        if self.boostrap:
            sample = self._subsample(dataset)

        self._tree = self._build_internal(sample)
        
        return self
    
    def predict(self, X):
        return np.array([self._rec_predict(self._tree, row) for row in X])
        
    def score(self, X, y):
        return np.sum(self.predict(X) == y) / float(len(y))
    
    # internal methods
    
    # Split a dataset based on an attribute and an attribute value
    def _test_split(self, index, value, dataset):
        feature = dataset[:, index]
        return dataset[feature < value], dataset[feature >= value]

    # Select the best split point for a dataset
    def _get_split(self, dataset):
        b_index, b_value, b_score, b_groups = 999, 999, 999, None

        features = self.random_state.choice(range(len(dataset[0])-1), self._num_features, replace=False)

        for index in features:
            for row in dataset:
                groups = self._test_split(index, row[index], dataset)
                score = self._score_f(groups)
                if score < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], score, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}

    # Create a terminal node value
    def _to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    # Create child splits for a node or make terminal
    def _split(self, node, depth):
        left, right = node['groups']
        del(node['groups'])
        # check for a no split
        if len(left) == 0 or len(right) == 0:
            node['left'] = node['right'] = self._to_terminal(np.vstack([left, right]))
            return
        # check for max depth
        if self.max_depth is not None and depth >= self.max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        # process left child
        if len(left) <= self.min_samples_leaf:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_split(left)
            self._split(node['left'], depth+1)
        # process right child
        if len(right) <= self.min_samples_leaf:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_split(right)
            self._split(node['right'], depth+1)

    # Make a prediction with a decision tree
    def _rec_predict(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self._rec_predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self._rec_predict(node['right'], row)
            else:
                return node['right']

    # Create a random subsample from the dataset with replacement
    def _subsample(self, dataset):
        n_sample = len(dataset)
        index = self.random_state.choice(range(n_sample), n_sample, replace=self.boostrap)
        sample = dataset[index]
        return sample 

    # Build a decision tree
    def _build_internal(self, train):
        root = self._get_split(train)
        self._split(root, 1)
        return root
    
class MyRandomForestClassifier(sklearn.base.BaseEstimator):

    def __init__(self, n_estimators=10, max_features=None, max_depth=None, min_samples_leaf=1, boostrap=True, criterion='gini', random_state=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.boostrap = boostrap
        self.criterion = criterion
        self.random_state = random_state
        
        # internal representation
        self._estimators = []
        
    def fit(self, X, y):
        # generate random numbers according to scikit guidelines
        self.random_state = sklearn.utils.check_random_state(self.random_state)        

        # default: sqrt of the total number of features
        num_features = int(np.sqrt(len(X[0]))) if self.max_features is None else self.max_features
            
        params = dict(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, 
                      max_features=num_features, boostrap=self.boostrap, criterion=self.criterion,
                      random_state=self.random_state) 

        self._estimators = [MyDecisionTreeClassifier(**params).fit(X, y) for t in range(self.n_estimators)]

        return self

    def predict(self, X):        
        return np.array([self._bagging_predict(sample) for sample in X]) 

    def score(self, X, y):
        return np.sum(self.predict(X) == y) / float(len(y))
    
    # Make a prediction with a list of bagged trees
    def _bagging_predict(self, sample):
        class_values, counts = np.unique([tree.predict([sample]) for tree in self._estimators], return_counts=True)
        return class_values[np.argmax(counts)]

<div class = "alert alert-success" style = "border-radius:10px">**WP 4**
<ol>
<li> Test your classifier.</li> 
</ol>
<div>

In [6]:
%time my_clf = MyRandomForestClassifier(n_estimators=10, criterion='entropy').fit(X_train, y_train)

Wall time: 2min 32s


In [10]:
%time predictions = my_clf.predict(X_test)

print('Confusion Matrix testing dataset')
text = """
            True class
              0   1
Predicted  0 {tn}  {fn} 
class      1 {fp}  {tp}       
"""

POS, NEG = 1, -1
TP = np.sum( y_test[predictions == POS] == POS )
TN = np.sum( y_test[predictions == NEG] == NEG )
FP = np.sum( y_test[predictions == POS] == NEG )
FN = np.sum( y_test[predictions == NEG] == POS )

print(text.format(tn=TN, tp=TP, fn=FN, fp=FP))
accuracy = 1. * (TN + TP) / (TN + TP + FN + FP)
precision = 1. * TP / (TP + FP)
recall = 1. * TP / (TP + FN)
print('Our Random Forest -- Stats') 
print('accuracy:{:.3f} precision:{:.3f} recall:{:.3f}'.format(accuracy, precision, recall))

Wall time: 105 ms
Confusion Matrix testing dataset

            True class
              0   1
Predicted  0 564  35 
class      1 2  66       

Our Random Forest -- Stats
accuracy:0.945 precision:0.971 recall:0.653


<div class = "alert alert-success" style = "border-radius:10px">**WP 5**
<ol>
<li> Evaluate your code comparing the results with a couple of techniques from sklearn.</li> 
</ol>
<div>

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost"]

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear"),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(criterion='entropy'),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
]

# iterate over classifiers
print('Classifier -- Testing accuracy')
for name, clf in zip(names, classifiers):
    %time clf.fit(X_train, y_train)
    %time accuracy = clf.score(X_test, y_test)
    print('{}: {:.3f}\n'.format(name, accuracy))

Classifier -- Testing accuracy
Wall time: 4 ms
Wall time: 84 ms
Nearest Neighbors: 0.894

Wall time: 112 ms
Wall time: 17 ms
Linear SVM: 0.849

Wall time: 125 ms
Wall time: 22 ms
RBF SVM: 0.930

Wall time: 30 ms
Wall time: 1e+03 µs
Decision Tree: 0.921

Wall time: 102 ms
Wall time: 8 ms
Random Forest: 0.946

Wall time: 843 ms
Wall time: 2 ms
Neural Net: 0.939

Wall time: 295 ms
Wall time: 7 ms
AdaBoost: 0.877



The problem is an unbalanced problem and our goal is to miss as few as possible
churn clients. For each predicted churn client the company will spend resources to try to
keep him engaged.
Consider that a given client yields a benefit of 100 euros.

<div class = "alert alert-success" style = "border-radius:10px">**WP 6**
<ol>
<li> Propose a solution for different costs (in euros) for regaining a client</li> 
<li> Look for the most profitable retention campaign (How much can be invest in
retention?)</li> 

</ol>
<div>

<div class = "alert alert-success" style = "border-radius:10px">**Results**
<ol>
<li> Table comparing performances. This includes, for instance, error rate and standard deviation. </li> 
<li> Graphical presentation of the information in the previous table. </li> 
<li> Graphical representation of the benefit vs the cost of the retention campaign. </li>
</ol>
<div>

## V. Conclusions and Future Work
todo
## VI. Bibliography
todo